# Preprocesamiento de Datos – ASTRO vs DM

## Separación de Datos ASTRO y DM para Modelado One-Class SVM

El archivo original `XY_bal_log_Rel.txt` contiene un conjunto de datos etiquetado, en el cual las instancias se clasifican como:

- **0 (ASTRO)**: fuentes astrofísicas reales, identificadas por telescopios.
- **1 (DM)**: fuentes simuladas que imitan la señal esperada de materia oscura (Dark Matter).

El dataset también contiene las siguientes variables (features):

1. `Log(E_peak)`: Logaritmo del pico de energía del espectro gamma.
2. `Log(beta)`: Logaritmo de la curvatura espectral.
3. `Log(sigma)`: Logaritmo de la significancia de detección.
4. `Log(beta_Rel)`: Logaritmo del error relativo de `beta`.

---

### Objetivo del preprocesamiento

El propósito principal de este notebook es preparar los datos para entrenar un modelo **One-Class SVM**, el cual está diseñado para aprender únicamente a partir de ejemplos de una sola clase, considerada "normal". En este caso:

> Se considera que las fuentes ASTRO representan el comportamiento normal observable en el espacio gamma.

Por tanto, se decide **separar el dataset original en dos subconjuntos distintos**:

- Un archivo que contiene exclusivamente fuentes ASTRO (`astro_DM == 0`), que será usado para el entrenamiento del modelo.
- Otro archivo que contiene las simulaciones de materia oscura (`astro_DM == 1`), que será reservado para futuras fases de evaluación del modelo.

---

### Procedimiento aplicado

1. Se cargó el dataset original con nombres de columnas definidos manualmente.
2. Se verificó la distribución de clases y se confirmó que los datos estaban correctamente etiquetados.
3. Se dividieron los datos en dos nuevos DataFrames: `astro_df` y `dm_df`.
4. Ambos subconjuntos se guardaron en rutas separadas para su posterior reutilización:

- `data/processed/XY_bal_log_Rel/astro/XY_bal_log_Rel_astro.txt`
- `data/processed/XY_bal_log_Rel/DM/XY_bal_log_Rel_DM.txt`

---

### Justificación metodológica

Separar las clases desde el preprocesamiento tiene varias ventajas:

- Permite un entrenamiento limpio del One-Class SVM con datos puramente normales.
- Evita introducir sesgos derivados de simulaciones (DM) en la fase de entrenamiento.
- Establece un flujo de datos reproducible y controlado, ideal para procesos de validación y experimentación.

Este enfoque está alineado con el objetivo general del trabajo: identificar potenciales señales de materia oscura a través de la detección de anomalías en fuentes no identificadas (unIDs), partiendo de un modelo entrenado únicamente con datos astrofísicos reales.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

In [ ]:
# Definir nombres de las columnas
features = ['E_peak', 'beta', 'sigma', 'beta_Rel', '0,1=astro,DM']

# Cargar los datos
df = pd.read_csv('../../data/raw/XY_bal_log_Rel.txt', sep="\s+", names=features, engine='python', skiprows=1)

# Renombrar la columna de etiquetas para mayor claridad
df = df.rename(columns={'0,1=astro,DM': 'astro_DM'})

print(f"Dimensiones del dataset: {df.shape}")
print(f"Número total de observaciones: {len(df)}")

# Verificación de datos

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
# Distribución de clases
print(df['astro_DM'].value_counts())


In [ ]:
# Estadísticas por clase
print(f"\n\nEstadísticas por clase:")

print(f"\nCLASE ASTRO (0.0):")
astro_stats = df[df['astro_DM'] == 0.0].describe()
print(astro_stats)

print(f"\nCLASE DM (1.0):")
dm_stats = df[df['astro_DM'] == 1.0].describe()
print(dm_stats)

In [ ]:
# Variables de características (excluyendo la etiqueta)
feature_cols = ['E_peak', 'beta', 'sigma', 'beta_Rel']

In [ ]:
# Histogramas superpuestos por clase
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.ravel()

for i, col in enumerate(feature_cols):
    # Datos por clase
    astro_data = df[df['astro_DM'] == 0.0][col]
    dm_data = df[df['astro_DM'] == 1.0][col]
    
    # Histogramas superpuestos
    axes[i].hist(astro_data, bins=40, alpha=0.6, color='skyblue', label='ASTRO', density=True)
    axes[i].hist(dm_data, bins=40, alpha=0.6, color='lightcoral', label='DM', density=True)
    
    axes[i].set_title(f'Distribución de {col} por Clase')
    axes[i].set_xlabel(col)
    axes[i].set_ylabel('Densidad')
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Crear scatter plots para pares de variables más interesantes
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.ravel()

# Pares de variables importantes
pairs = [
    ('E_peak', 'beta'),
    ('E_peak', 'sigma'),
    ('beta', 'sigma'),
    ('beta', 'beta_Rel'),
    ('sigma', 'beta_Rel'),
    ('E_peak', 'beta_Rel')
]

for i, (x_var, y_var) in enumerate(pairs):
    # Separar por clase
    astro_data = df[df['astro_DM'] == 0.0]
    dm_data = df[df['astro_DM'] == 1.0]
    
    # Scatter plot
    axes[i].scatter(astro_data[x_var], astro_data[y_var], alpha=0.6, 
                   c='skyblue', label='ASTRO', s=20)
    axes[i].scatter(dm_data[x_var], dm_data[y_var], alpha=0.6, 
                   c='lightcoral', label='DM', s=20)
    
    axes[i].set_xlabel(x_var)
    axes[i].set_ylabel(y_var)
    axes[i].set_title(f'{x_var} vs {y_var}')
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Preprocesamiento

In [ ]:
# Separar datos por clase
astro_df = df[df['astro_DM'] == 0.0].copy()
dm_df = df[df['astro_DM'] == 1.0].copy()

print(f"Datos ASTRO: {len(astro_df)} observaciones")
print(f"Datos DM: {len(dm_df)} observaciones")

# Mostrar estadísticas finales de cada conjunto
print("\nEstadísticas finales - Conjunto ASTRO:")
print(astro_df[feature_cols].describe())

print("\nEstadísticas finales - Conjunto DM:")
print(dm_df[feature_cols].describe())

In [ ]:
# Crear nombres de columnas con prefijo Log() para indicar transformación logarítmica
log_feature_names = ['Log(E_peak)', 'Log(beta)', 'Log(sigma)', 'Log(beta_Rel)', 'astro_DM']

# Preparar datasets con nombres de columnas actualizados
astro_df_save = astro_df.copy()
dm_df_save = dm_df.copy()
df_complete_save = df.copy()

# Renombrar columnas para indicar transformación logarítmica
column_mapping = dict(zip(['E_peak', 'beta', 'sigma', 'beta_Rel', 'astro_DM'], log_feature_names))
astro_df_save = astro_df_save.rename(columns=column_mapping)
dm_df_save = dm_df_save.rename(columns=column_mapping)
df_complete_save = df_complete_save.rename(columns=column_mapping)

print("Nombres de columnas actualizados:")
for old_name, new_name in column_mapping.items():
    print(f"  {old_name} → {new_name}")

In [ ]:
print(f"\nGuardando datos ASTRO:")

# Con etiqueta - CSV
astro_df_save.to_csv('../../data/processed/XY_bal_log_Rel/astro/astro_data_with_labels.csv', index=False)
print("- astro_data_with_labels.csv")

# Con etiqueta - TXT
astro_df_save.to_csv('../../data/processed/XY_bal_log_Rel/astro/astro_data_with_labels.txt', sep='\t', index=False)
print("- astro_data_with_labels.txt")

# Sin etiqueta (solo features) - CSV
astro_features_only = astro_df_save[log_feature_names[:-1]]  # Excluir astro_DM
astro_features_only.to_csv('../../data/processed/XY_bal_log_Rel/astro/astro_data.csv', index=False)
print("- astro_data.csv (solo features)")

# Sin etiqueta (solo features) - TXT
astro_features_only.to_csv('../../data/processed/XY_bal_log_Rel/astro/astro_data.txt', sep='\t', index=False)
print("- astro_data.txt (solo features)")

In [ ]:
print(f"\nGuardando datos DM:")

# Con etiqueta - CSV
dm_df_save.to_csv('../../data/processed/XY_bal_log_Rel/DM/dm_data_with_labels.csv', index=False)
print("- dm_data_with_labels.csv")

# Con etiqueta - TXT
dm_df_save.to_csv('../../data/processed/XY_bal_log_Rel/DM/dm_data_with_labels.txt', sep='\t', index=False)
print("- dm_data_with_labels.txt")

# Sin etiqueta (solo features) - CSV
dm_features_only = dm_df_save[log_feature_names[:-1]]  # Excluir astro_DM
dm_features_only.to_csv('../../data/processed/XY_bal_log_Rel/DM/dm_data.csv', index=False)
print("- dm_data.csv (solo features)")

# Sin etiqueta (solo features) - TXT
dm_features_only.to_csv('../../data/processed/XY_bal_log_Rel/DM/dm_data.txt', sep='\t', index=False)
print("- dm_data.txt (solo features)")

In [ ]:
print(f"\nGuardando dataset completo:")

# Dataset completo - CSV
df_complete_save.to_csv('../../data/processed/XY_bal_log_Rel/fermi_lat_complete.csv', index=False)
print("- fermi_lat_complete.csv")

# Dataset completo - TXT
df_complete_save.to_csv('../../data/processed/XY_bal_log_Rel/fermi_lat_complete.txt', sep='\t', index=False)
print("- fermi_lat_complete.txt")

# Reverificación de datos

In [ ]:
try:
    # Cargar datos ASTRO desde TXT
    print("Cargando astro_data_with_labels.txt...")
    astro_reloaded = pd.read_csv('../../data/processed/XY_bal_log_Rel/astro/astro_data_with_labels.txt', sep='\t')
    print(f"- Archivo cargado exitosamente: {astro_reloaded.shape}")
    
    # Verificar estructura
    print(f"\nESTRUCTURA DEL DATASET ASTRO:")
    print(f"  • Dimensiones: {astro_reloaded.shape}")
    print(f"  • Columnas: {list(astro_reloaded.columns)}")
    print(f"  • Tipos de datos:")
    for col, dtype in astro_reloaded.dtypes.items():
        print(f"    - {col}: {dtype}")
    
    # Verificar contenido
    print(f"\nCONTENIDO DEL DATASET ASTRO:")
    print("Primeras 5 filas:")
    print(astro_reloaded.head())
    
    print("\nÚltimas 3 filas:")
    print(astro_reloaded.tail(3))
    
    # Verificar etiquetas
    print(f"\nVERIFICACIÓN DE ETIQUETAS:")
    print(f"  • Valores únicos en astro_DM: {astro_reloaded['astro_DM'].unique()}")
    print(f"  • Conteo de clases: {astro_reloaded['astro_DM'].value_counts().to_dict()}")
    
    # Verificar integridad
    original_astro_count = len(astro_df_save)
    reloaded_astro_count = len(astro_reloaded)
    
    print(f"\nVERIFICACIÓN DE INTEGRIDAD ASTRO:")
    print(f"  • Observaciones originales: {original_astro_count:,}")
    print(f"  • Observaciones recargadas: {reloaded_astro_count:,}")
    
    if original_astro_count == reloaded_astro_count:
        print("Sin pérdida de datos")
    else:
        print("Posible pérdida de datos")
    
    # Verificar nombres de columnas
    expected_columns = ['Log(E_peak)', 'Log(beta)', 'Log(sigma)', 'Log(beta_Rel)', 'astro_DM']
    
    print(f"\nVERIFICACIÓN DE NOMBRES DE COLUMNAS:")
    print(f"  • Esperadas: {expected_columns}")
    print(f"  • Encontradas: {list(astro_reloaded.columns)}")
    
    if list(astro_reloaded.columns) == expected_columns:
        print("Nombres de columnas correctos")
    else:
        print("Nombres de columnas no coinciden")
    
    # Estadísticas básicas
    feature_columns = ['Log(E_peak)', 'Log(beta)', 'Log(sigma)', 'Log(beta_Rel)']
    print(f"\nESTADÍSTICAS DESCRIPTIVAS ASTRO:")
    print(astro_reloaded[feature_columns].describe())
    
    print("\n" + "="*60)
    print("VERIFICACIÓN ASTRO COMPLETADA EXITOSAMENTE")
    print("="*60)
    
except FileNotFoundError as e:
    print(f"Error: No se encontró astro_data_with_labels.txt")
    print(f"Detalle: {e}")
    
except Exception as e:
    print(f"Error inesperado al verificar datos ASTRO: {e}")

In [ ]:
try:
    # Cargar datos DM desde TXT  
    print("Cargando dm_data_with_labels.txt...")
    dm_reloaded = pd.read_csv('../../data/processed/XY_bal_log_Rel/DM/dm_data_with_labels.txt', sep='\t')
    print(f"- Archivo cargado exitosamente: {dm_reloaded.shape}")
    
    # Verificar estructura
    print(f"\nESTRUCTURA DEL DATASET DM:")
    print(f"  • Dimensiones: {dm_reloaded.shape}")
    print(f"  • Columnas: {list(dm_reloaded.columns)}")
    print(f"  • Tipos de datos:")
    for col, dtype in dm_reloaded.dtypes.items():
        print(f"    - {col}: {dtype}")
    
    # Verificar contenido
    print(f"\nCONTENIDO DEL DATASET DM:")
    print("Primeras 5 filas:")
    print(dm_reloaded.head())
    
    print("\nÚltimas 3 filas:")
    print(dm_reloaded.tail(3))
    
    # Verificar etiquetas
    print(f"\nVERIFICACIÓN DE ETIQUETAS:")
    print(f"  • Valores únicos en astro_DM: {dm_reloaded['astro_DM'].unique()}")
    print(f"  • Conteo de clases: {dm_reloaded['astro_DM'].value_counts().to_dict()}")
    
    # Verificar integridad
    original_dm_count = len(dm_df_save)
    reloaded_dm_count = len(dm_reloaded)
    
    print(f"\nVERIFICACIÓN DE INTEGRIDAD DM:")
    print(f"  • Observaciones originales: {original_dm_count:,}")
    print(f"  • Observaciones recargadas: {reloaded_dm_count:,}")
    
    if original_dm_count == reloaded_dm_count:
        print("Sin pérdida de datos")
    else:
        print("Posible pérdida de datos")
    
    # Verificar nombres de columnas
    expected_columns = ['Log(E_peak)', 'Log(beta)', 'Log(sigma)', 'Log(beta_Rel)', 'astro_DM']
    
    print(f"\nVERIFICACIÓN DE NOMBRES DE COLUMNAS:")
    print(f"  • Esperadas: {expected_columns}")
    print(f"  • Encontradas: {list(dm_reloaded.columns)}")
    
    if list(dm_reloaded.columns) == expected_columns:
        print("Nombres de columnas correctos")
    else:
        print("Nombres de columnas no coinciden")
    
    # Estadísticas básicas
    feature_columns = ['Log(E_peak)', 'Log(beta)', 'Log(sigma)', 'Log(beta_Rel)']
    print(f"\nESTADÍSTICAS DESCRIPTIVAS DM:")
    print(dm_reloaded[feature_columns].describe())
    
    print("\n" + "="*60)
    print("VERIFICACIÓN DM COMPLETADA EXITOSAMENTE")
    print("="*60)
    
except FileNotFoundError as e:
    print(f"Error: No se encontró dm_data_with_labels.txt")
    print(f"Detalle: {e}")
    
except Exception as e:
    print(f"Error inesperado al verificar datos DM: {e}")